In [1]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

from config.experiment_config import ExperimentConfig

config = ExperimentConfig()

In [3]:
market_data = pd.read_excel(config.PATH_INPUT / config.INPUT_DATA_FILENAME)
market_data = market_data.rename(columns={"Date": "date"})
market_data["date"] = pd.to_datetime(market_data["date"])
market_data = market_data.set_index("date")
market_data.head()

,EMP,PE,CAPE,DY,Rho,MOV,IR,RR,Y02,Y10,...,YSS,NYF,_AU,_DXY,_LCP,_TY,_OIL,_MKT,_VA,_GR
date,,,,,,,,,,,,,,,,,,,,,
1988-04-10,0.022086,12.9,14.469,3.60,-0.083,118.0,6.02,-1.061,7.459,8.484,...,0.248874,6.896895,450.50,89.14,2473.7,97.990,16.88,419.06,823.219,462.551
1988-04-17,0.022086,12.4,13.960,3.75,-0.078,121.0,5.88,-0.760,7.582,8.737,...,0.326712,2.631749,456.25,88.31,2328.2,96.534,18.37,404.66,823.219,462.551
1988-04-24,0.022086,12.4,13.950,3.75,-0.051,123.0,5.83,-0.760,7.618,8.773,...,0.155089,2.631749,449.25,88.89,2201.4,96.470,18.30,404.71,823.219,462.551
1988-05-01,0.021941,12.5,14.036,3.75,-0.054,124.2,5.98,-0.760,7.728,8.873,...,0.145360,2.631749,449.00,89.16,2147.8,96.005,18.00,407.55,840.891,461.176
1988-05-08,0.022104,12.3,13.761,3.82,-0.079,118.4,6.29,-0.760,7.885,8.990,...,0.146644,2.631749,443.00,89.30,2200.9,95.446,17.74,401.29,840.891,461.176


In [4]:
market_data["ts"] = market_data["Y10"] - market_data["Y02"]

In [5]:
from ipsqt.market_data.risk_free_conventions import accrue_risk_free_rate

In [6]:
data = market_data[["CAPE", "DY", "ts", "IR", "_OIL", "_MKT"]]
data["_MKT"] = data["_MKT"].pct_change()
data["IR"] = data["IR"] / 100 / 360
data["IR"] = accrue_risk_free_rate(data["IR"])
data[data.columns.difference(["_MKT", "ts", "IR"])] = np.log(
    data[data.columns.difference(["_MKT", "ts", "IR"])]
)
data[data.columns.difference(["_MKT", "ts", "IR"])] = data[
    data.columns.difference(["_MKT", "ts", "IR"])
] - data[data.columns.difference(["_MKT", "ts", "IR"])].shift(1)
data

/var/folders/sr/lzvmv9j54ks_jl4x99t250r00000gn/T/ipykernel_4173/266021148.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["_MKT"] = data["_MKT"].pct_change()
/var/folders/sr/lzvmv9j54ks_jl4x99t250r00000gn/T/ipykernel_4173/266021148.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["IR"] = data["IR"] / 100 / 360
/var/folders/sr/lzvmv9j54ks_jl4x99t250r00000gn/T/ipykernel_4173/266021148.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,CAPE,DY,ts,IR,_OIL,_MKT
date,,,,,,
1988-04-10,NaN,NaN,1.025,NaN,NaN,NaN
1988-04-17,-0.035812,0.040822,1.155,0.001143,0.084589,-0.034363
1988-04-24,-0.000717,0.000000,1.155,0.001134,-0.003818,0.000124
1988-05-01,0.006146,0.000000,1.145,0.001163,-0.016529,0.007017
1988-05-08,-0.019787,0.018495,1.105,0.001223,-0.014550,-0.015360
...,...,...,...,...,...,...
2024-04-07,-0.010370,0.014599,-0.417,0.001043,0.043467,-0.009351
2024-04-14,-0.017253,0.014389,-0.447,0.001047,-0.014126,-0.016173
2024-04-21,-0.032073,0.035091,-0.425,0.001049,-0.031368,-0.030627


In [7]:
data.iloc[1:].to_csv(config.PATH_OUTPUT / config.DF_FILENAME)

In [8]:
market_data["_MKT"].apply(lambda x: 1).to_csv(
    config.PATH_OUTPUT / config.PRESENCE_MATRIX_FILENAME
)